#### Imports

In [44]:
import pyspark
from pyspark.sql import SparkSession
from IPython.display import display
from pyspark.sql.functions import udf ,col , split, max, min, sum,year,month,dayofmonth,weekofyear
from pyspark.sql.types import DateType,StringType
from datetime import datetime

# Basic operations

In [45]:
spark = SparkSession.builder.appName("stocksly").getOrCreate()
spark

In [50]:
stocks = spark.read.csv(
    "../data_sourcing/source/stocks.csv", 
    header=True, 
    inferSchema=True
)

In [51]:
stocks = stocks.limit(10000)

In [52]:
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|
+---+---------+-------------------+-----+-----+-----+-----+------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|
+---+---------+-------------------+-----+-----+-----+-----+------+
only showing top 5 rows



25/02/20 01:23:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [54]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- stockname: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)



In [55]:
stocks.select("stockname").show(5)

+---------+
|stockname|
+---------+
|    PLTR |
|    PLTR |
|    PLTR |
|    PLTR |
|    PLTR |
+---------+
only showing top 5 rows



In [56]:
stocks.select(
    ["stockname","open", "close"]
).show(5)

+---------+-----+-----+
|stockname| open|close|
+---------+-----+-----+
|    PLTR |81.94|81.44|
|    PLTR |81.39|82.01|
|    PLTR |81.96|82.13|
|    PLTR |82.19|82.38|
|    PLTR | 82.4|82.32|
+---------+-----+-----+
only showing top 5 rows



In [57]:
stocks.filter(
    stocks.stockname == "TSLA"
).show(5)

25/02/20 01:23:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+---+---------+---------+----+----+---+-----+------+
|_c0|stockname|timestamp|open|high|low|close|volume|
+---+---------+---------+----+----+---+-----+------+
+---+---------+---------+----+----+---+-----+------+



In [58]:
stocks.filter(
    (
        (stocks.stockname == "TSLA") |
        (stocks.stockname == "PLTR")
    ) & (
    stocks.timestamp == "2024-12-23 09:00:00")
).show()

25/02/20 01:23:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+---+---------+---------+----+----+---+-----+------+
|_c0|stockname|timestamp|open|high|low|close|volume|
+---+---------+---------+----+----+---+-----+------+
+---+---------+---------+----+----+---+-----+------+



In [59]:
stocks.filter(
    stocks.stockname.isin("TSLA", "PLTR") &
    (stocks.timestamp == "2024-12-23 09:00:00")
).show()

25/02/20 01:23:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+---+---------+---------+----+----+---+-----+------+
|_c0|stockname|timestamp|open|high|low|close|volume|
+---+---------+---------+----+----+---+-----+------+
+---+---------+---------+----+----+---+-----+------+



In [60]:
stocks = stocks.withColumn(
    "date", split(
        col("timestamp"), 
    " ")[0]
    )
stocks = stocks.withColumn(
    "time", split(
        col("timestamp"), 
    " ")[1]
    )
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
only showing top 5 rows



25/02/20 01:23:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [61]:
date_parser = udf(
    lambda date : datetime.strptime(date, "%Y-%m-%d"),DateType()
)

In [62]:
stocks = stocks.withColumn(
    "new_date",
    date_parser(stocks.date)  
)

stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/20 01:23:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [63]:
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/20 01:23:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [64]:
def upper_turner(stockname):
    return stockname.upper()

upper_turner = udf(
    upper_turner, StringType()
)

In [65]:
stocks = stocks.withColumn(
    "stockname",upper_turner(stocks.stockname)
)
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/20 01:23:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [66]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- stockname: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- new_date: date (nullable = true)



In [67]:
new_stocks = stocks.select(
    "stockname",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "new_date",
    "time"
)

In [68]:
new_stocks.show(5)

+---------+-----+-----+-----+-----+------+----------+--------+
|stockname| open| high|  low|close|volume|  new_date|    time|
+---------+-----+-----+-----+-----+------+----------+--------+
|    PLTR |81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|
|    PLTR |81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|
|    PLTR |81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|
|    PLTR |82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|
|    PLTR | 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|
+---------+-----+-----+-----+-----+------+----------+--------+
only showing top 5 rows



In [69]:
new_stocks.describe().show()

+-------+---------+------------------+------------------+------------------+------------------+------+--------+
|summary|stockname|              open|              high|               low|             close|volume|    time|
+-------+---------+------------------+------------------+------------------+------------------+------+--------+
|  count|    10000|              9931|              9931|              9931|              9931|     0|   10000|
|   mean|     NULL|217.83492173662867|218.03004032644918|217.61086120902064| 217.8328568690507|  NULL|    NULL|
| stddev|     NULL|145.04690589319335|145.19137065095407|144.87971120522596|145.04797447371956|  NULL|    NULL|
|    min|    MSTR |  74.7699966430664|  74.8088989257813|             64.35|             74.75|  NULL|00:00:00|
|    max|    TSLA |            436.67|           468.475|            436.59|            436.79|  NULL|23:59:00|
+-------+---------+------------------+------------------+------------------+------------------+------+--

# Basic stock analysis

In [70]:
new_stocks.groupBy("stockname").max("Open").show(15)

+---------+----------------+
|stockname|       max(Open)|
+---------+----------------+
|    PLTR |84.6800003051758|
|    MSTR |          365.47|
|    ROKU |            81.6|
|    TSLA |          436.67|
+---------+----------------+



In [71]:
new_stocks.groupBy("stockname").max("Open").withColumnRenamed("max(Open)", "MaxStockPrice").show(15)

+---------+----------------+
|stockname|   MaxStockPrice|
+---------+----------------+
|    PLTR |84.6800003051758|
|    MSTR |          365.47|
|    ROKU |            81.6|
|    TSLA |          436.67|
+---------+----------------+



In [72]:
new_stocks.groupBy("stockname").agg(max("Open").alias("MaxStockPrice")).show(15)

+---------+----------------+
|stockname|   MaxStockPrice|
+---------+----------------+
|    PLTR |84.6800003051758|
|    MSTR |          365.47|
|    ROKU |            81.6|
|    TSLA |          436.67|
+---------+----------------+



In [73]:
new_stocks.groupBy("stockname").agg(
    max("open").alias("MaxStockPrice"),
    sum("volume").alias("TotalVolume")
).show(15)

+---------+----------------+-----------+
|stockname|   MaxStockPrice|TotalVolume|
+---------+----------------+-----------+
|    PLTR |84.6800003051758|       NULL|
|    MSTR |          365.47|       NULL|
|    ROKU |            81.6|       NULL|
|    TSLA |          436.67|       NULL|
+---------+----------------+-----------+



In [74]:
new_stocks = (
    new_stocks.withColumn("Year", year(new_stocks.new_date))
    .withColumn("Month", month(new_stocks.new_date))
    .withColumn("Day", dayofmonth(new_stocks.new_date))
    .withColumn("Week", weekofyear(new_stocks.new_date))
)

In [75]:
new_stocks.show(5)

+---------+-----+-----+-----+-----+------+----------+--------+----+-----+---+----+
|stockname| open| high|  low|close|volume|  new_date|    time|Year|Month|Day|Week|
+---------+-----+-----+-----+-----+------+----------+--------+----+-----+---+----+
|    PLTR |81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024|   12| 23|  52|
|    PLTR |81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024|   12| 23|  52|
|    PLTR |81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024|   12| 23|  52|
|    PLTR |82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024|   12| 23|  52|
|    PLTR | 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024|   12| 23|  52|
+---------+-----+-----+-----+-----+------+----------+--------+----+-----+---+----+
only showing top 5 rows



In [76]:
yearly = new_stocks.groupBy(
    ['stockname', 'Year']
).agg(
    max("open").alias("YearlHigh"), 
    min("open").alias("YearlyLow")
)
yearly.show(5)

+---------+----+----------------+----------------+
|stockname|Year|       YearlHigh|       YearlyLow|
+---------+----+----------------+----------------+
|    PLTR |2024|84.6800003051758|77.4250030517578|
|    MSTR |2024|          365.47|323.372711181641|
|    ROKU |2024|            81.6|74.7699966430664|
|    TSLA |2024|          436.67|416.929992675781|
+---------+----+----------------+----------------+



In [77]:
monthly = new_stocks.groupBy(
    ["stockname", "year", "month"]  # Ensure lowercase if needed
).agg(
    max("Open").alias("MonthHigh"), 
    min("Open").alias("MonthLow")
)
monthly.show(5)

+---------+----+-----+----------------+----------------+
|stockname|year|month|       MonthHigh|        MonthLow|
+---------+----+-----+----------------+----------------+
|    PLTR |2024|   12|84.6800003051758|77.4250030517578|
|    MSTR |2024|   12|          365.47|323.372711181641|
|    ROKU |2024|   12|            81.6|74.7699966430664|
|    TSLA |2024|   12|          436.67|416.929992675781|
+---------+----+-----+----------------+----------------+



In [78]:
weekly = new_stocks.groupBy(
    ['stockname', 'Year', 'Week']
).agg(
    max("Open").alias("WeekHigh"), min("Open").alias("WeekLow")
)
weekly.show(5)

+---------+----+----+----------------+----------------+
|stockname|Year|Week|        WeekHigh|         WeekLow|
+---------+----+----+----------------+----------------+
|    PLTR |2024|  52|84.6800003051758|77.4250030517578|
|    MSTR |2024|  52|          365.47|323.372711181641|
|    ROKU |2024|  52|            81.6|74.7699966430664|
|    TSLA |2024|  52|          436.67|416.929992675781|
+---------+----+----+----------------+----------------+



In [79]:
weekly = weekly.withColumn(
    "Spread", weekly['WeekHigh'] - weekly['WeekLow']
)
weekly.show(5)

+---------+----+----+----------------+----------------+------------------+
|stockname|Year|Week|        WeekHigh|         WeekLow|            Spread|
+---------+----+----+----------------+----------------+------------------+
|    PLTR |2024|  52|84.6800003051758|77.4250030517578| 7.254997253417997|
|    MSTR |2024|  52|          365.47|323.372711181641|42.097288818359004|
|    ROKU |2024|  52|            81.6|74.7699966430664| 6.830003356933588|
|    TSLA |2024|  52|          436.67|416.929992675781|19.740007324218993|
+---------+----+----+----------------+----------------+------------------+



# Joins

In [80]:
historic_stocks = new_stocks.join(
    yearly, 
    (
        new_stocks.stockname==yearly.stockname
    ) & (
        new_stocks.Year == yearly.Year
        ),
    'inner'
).drop(
    yearly.Year, 
    yearly.stockname
)

In [81]:
historic_stocks.show(5)

+-----+-----+-----+-----+------+----------+--------+-----+---+----+---------+----+----------------+----------------+
| open| high|  low|close|volume|  new_date|    time|Month|Day|Week|stockname|Year|       YearlHigh|       YearlyLow|
+-----+-----+-----+-----+------+----------+--------+-----+---+----+---------+----+----------------+----------------+
|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|   12| 23|  52|    PLTR |2024|84.6800003051758|77.4250030517578|
|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|   12| 23|  52|    PLTR |2024|84.6800003051758|77.4250030517578|
|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|   12| 23|  52|    PLTR |2024|84.6800003051758|77.4250030517578|
|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|   12| 23|  52|    PLTR |2024|84.6800003051758|77.4250030517578|
| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|   12| 23|  52|    PLTR |2024|84.6800003051758|77.4250030517578|
+-----+-----+-----+-----+------+----------+--------+-----+---+--

In [82]:
cond = [
    (
        historic_stocks.stockname==weekly.stockname
    ) & (
        historic_stocks.Year == weekly.Year
    ) & (
        historic_stocks.Week == weekly.Week
    )
]
historic_stocks = historic_stocks.join(
    weekly, 
    cond, 
    'inner'
).drop(
    weekly.Year, 
    historic_stocks.stockname, 
    weekly.Week
)

In [83]:
historic_stocks.show(5)

+-----+-----+-----+-----+------+----------+--------+-----+---+----+----------------+----------------+---------+----+----------------+----------------+-----------------+
| open| high|  low|close|volume|  new_date|    time|Month|Day|Year|       YearlHigh|       YearlyLow|stockname|Week|        WeekHigh|         WeekLow|           Spread|
+-----+-----+-----+-----+------+----------+--------+-----+---+----+----------------+----------------+---------+----+----------------+----------------+-----------------+
|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|   12| 23|2024|84.6800003051758|77.4250030517578|    PLTR |  52|84.6800003051758|77.4250030517578|7.254997253417997|
|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|   12| 23|2024|84.6800003051758|77.4250030517578|    PLTR |  52|84.6800003051758|77.4250030517578|7.254997253417997|
|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|   12| 23|2024|84.6800003051758|77.4250030517578|    PLTR |  52|84.6800003051758|77.4250030517578|7.254

In [84]:
historic_stocks.columns

['open',
 'high',
 'low',
 'close',
 'volume',
 'new_date',
 'time',
 'Month',
 'Day',
 'Year',
 'YearlHigh',
 'YearlyLow',
 'stockname',
 'Week',
 'WeekHigh',
 'WeekLow',
 'Spread']

In [85]:
historic_stocks = historic_stocks.join(
    monthly, [
        'stockname', 
        'Year', 
        'Month'
    ]
)
historic_stocks.show()

+---------+----+-----+-----+-----+-----+-----+------+----------+--------+---+----------------+----------------+----+----------------+----------------+-----------------+----------------+----------------+
|stockname|Year|Month| open| high|  low|close|volume|  new_date|    time|Day|       YearlHigh|       YearlyLow|Week|        WeekHigh|         WeekLow|           Spread|       MonthHigh|        MonthLow|
+---------+----+-----+-----+-----+-----+-----+------+----------+--------+---+----------------+----------------+----+----------------+----------------+-----------------+----------------+----------------+
|    PLTR |2024|   12|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00| 23|84.6800003051758|77.4250030517578|  52|84.6800003051758|77.4250030517578|7.254997253417997|84.6800003051758|77.4250030517578|
|    PLTR |2024|   12|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00| 23|84.6800003051758|77.4250030517578|  52|84.6800003051758|77.4250030517578|7.254997253417997|84.6800003051758|77.

In [86]:
final_stocks = historic_stocks.select(
    [
        'stockname', 
        'Year',
        'Month', 
        'Day', 
        'Week', 
        'Volume',
        'Open',
        'Low',
        'High',
        'Close', 
        'YearlHigh',
        'YearlyLow',
        'WeekHigh',
        'WeekLow',
        'MonthHigh',
        'MonthLow'
    ]
)

In [87]:
final_stocks.show(5)

+---------+----+-----+---+----+------+-----+-----+-----+-----+----------------+----------------+----------------+----------------+----------------+----------------+
|stockname|Year|Month|Day|Week|Volume| Open|  Low| High|Close|       YearlHigh|       YearlyLow|        WeekHigh|         WeekLow|       MonthHigh|        MonthLow|
+---------+----+-----+---+----+------+-----+-----+-----+-----+----------------+----------------+----------------+----------------+----------------+----------------+
|    PLTR |2024|   12| 23|  52|  NULL|81.94|81.16|81.95|81.44|84.6800003051758|77.4250030517578|84.6800003051758|77.4250030517578|84.6800003051758|77.4250030517578|
|    PLTR |2024|   12| 23|  52|  NULL|81.39|81.37|82.02|82.01|84.6800003051758|77.4250030517578|84.6800003051758|77.4250030517578|84.6800003051758|77.4250030517578|
|    PLTR |2024|   12| 23|  52|  NULL|81.96|81.95|82.16|82.13|84.6800003051758|77.4250030517578|84.6800003051758|77.4250030517578|84.6800003051758|77.4250030517578|
|    PLTR 

In [88]:
final_stocks.createOrReplaceTempView('stockData')

In [90]:
spark.sql("SELECT * FROM stockData where stockname='MSFT' and Year='2023'").show(5)

+---------+----+-----+---+----+------+----+---+----+-----+---------+---------+--------+-------+---------+--------+
|stockname|Year|Month|Day|Week|Volume|Open|Low|High|Close|YearlHigh|YearlyLow|WeekHigh|WeekLow|MonthHigh|MonthLow|
+---------+----+-----+---+----+------+----+---+----+-----+---------+---------+--------+-------+---------+--------+
+---------+----+-----+---+----+------+----+---+----+-----+---------+---------+--------+-------+---------+--------+



# end

# continue